In [2]:
import sys; sys.path.insert(0,'../../yapmap')
import sys; sys.path.insert(0,'..')
from lltk import *
from lltk.model.networks import draw_nx
gdb=GDB()
# res=Corpus('markmark').sync(), Corpus('chicago').sync()

In [3]:
addrs=['_markmark/James,_Henry.The_Ambassadors', '_markmark/James,_Henry.The_Golden_Bowl', '_markmark/James,_Henry.The_Portrait_of_a_Lady', '_markmark/James,_Henry.The_Wings_of_the_Dove', '_markmark/James,_Henry.The_Bostonians', '_chicago/00000086', '_chicago/00000524', '_chicago/00000711', '_chicago/00000901', '_chicago/00000902', '_chicago/00001457', '_chicago/00001601', '_chicago/00001757', '_chicago/00002238', '_chicago/00002499', '_chicago/00003859', '_chicago/00000038', '_chicago/00003858', '_chicago/00004205', '_chicago/00000039', '_chicago/00000381', '_chicago/00000037', '_chicago/00000130', '_chicago/00001312', '_chicago/00002099', '_chicago/00003255']
texts=list(map(Text,addrs))
t=texts[0]
# t, t._meta

In [4]:
with log: t.init(remote=True)

[20:49:53] lltk.text.text.init( Init metadata: TextMarkMark(_markmark/James,_Henry.The_Ambassadors), kwargs = {} )
[20:49:53] lltk.text.text.get_remote_sources( TextMarkMark(_markmark/James,_Henry.The_Ambassadors) )
[20:49:53] lltk.corpus.corpus.Corpus( <- id = hathi )
[20:49:53] lltk.corpus.utils.load_corpus( <- id = hathi )
[20:49:53] lltk.corpus.utils.load_corpus( Importing corpus class "Hathi" from /Users/ryan/github/lltk/lltk/corpus/hathi/hathi.py )
[20:49:53] lltk.corpus.corpus.__init__( Hathi(hathi) )
[20:49:53] lltk.corpus.utils.load_corpus( -> HathiCorpus(hathi) )
[20:49:53] lltk.corpus.corpus.Corpus( -> HathiCorpus(hathi) )
[20:49:53] lltk.text.text.get_remote_sources( remote corpus: HathiCorpus(hathi) (TextMarkMark(_markmark/James,_Henry.The_Ambassadors)) )
[20:49:53] hathi.query_for_ids( <- TextMarkMark(_markmark/James,_Henry.The_Ambassadors) )
[20:49:53] hathi.query_for_ids( <- https://catalog.hathitrust.org/Search/Home?adv=1&lookfor%5B%5D=James&type%5B%5D=author&bool%5B%5

In [ ]:
t.title, [tx.title for tx in t.sources]

In [ ]:
for t in get_tqdm(texts): t.init(remote=True)

In [ ]:
stop

In [ ]:
tx=random.choice(texts)
tx, tx._meta

In [ ]:
pladys = [t for t in texts if 'portrait of a lady' in t.title.lower()]
t = pladys[0]
t

In [ ]:
def to_nx2(self,texts,rel=None,min_overlap=3,**kwargs):
    g=nx.Graph()
    all_texts = {t for t in texts} | {src for t in texts for src in t.sources}
    if log: log(f'all_texts = {len(all_texts)}')
    for t1 in get_tqdm(all_texts):
        for t2 in all_texts:
            if t1.addr>=t2.addr: continue
            overlap=t1.linked() & t2.linked()
            log(f'{t1} --> {t2}, {len(overlap)} {pf(overlap)}')
            if len(overlap) >= min_overlap:
                g.add_edge(t1.addr,t2.addr)
    return g

LLDBGraph.to_nx2=to_nx2


def to_nx(self,texts,rel=None,**kwargs):
    g=nx.MultiGraph()
    for tx in get_tqdm(texts):
        for src in tx.sources:
            for u,v,d in src.gdb.get_edges(src.addr):
                relx=d.get('rel')
                if rel and relx != rel: continue
                if not g.has_node(u): g.add_node(u)#,**Text(u).meta)
                if not g.has_node(v): g.add_node(v)#,**Text(v).meta)
                if not g.has_edge(u,v,key=relx):
                    g.add_edge(u,v,key=relx,**d)
    return g

LLDBGraph.to_nx=to_nx

In [ ]:
C=Corpus('markmark')

In [ ]:
Ctexts=set(texts)
len(Ctexts), Ctexts

In [ ]:
g=gdb.to_nx(Ctexts)
g

In [ ]:
from cdlib.algorithms import louvain

coms = louvain(g, resolution=1, randomize=False)
coms

In [ ]:
len(coms.communities)

In [ ]:
for nset in coms.communities:
    tset=list(map(Text,nset))
    print(Counter(t.title for t in tset if t.title))
    print(tset)
    print()